<a href="https://colab.research.google.com/github/PratiteeMalakar/hello-world/blob/main/attempt2fewshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Generate synthetic sales data
np.random.seed(42)
date_range = pd.date_range(start='2023-01-01', end='2023-12-31')
products = ['Product_A', 'Product_B', 'Product_C']
data = []

for product in products:
    sales = np.random.poisson(lam=20, size=len(date_range))  # Simulate sales with a Poisson distribution
    for date, sale in zip(date_range, sales):
        data.append([date, product, sale])

sales_df = pd.DataFrame(data, columns=['Date', 'Product', 'Sales'])

# Generate synthetic logistics data
# Additional synthetic data for inventory management and supplier performance

# Inventory data
inventory_data = {
    'Product': ['Product_A', 'Product_B', 'Product_C'],
    'Current_Stock': [500, 300, 800],
    'Reorder_Level': [200, 100, 300],
    'Lead_Time_Days': [5, 7, 3]
}
inventory_df = pd.DataFrame(inventory_data)

# Supplier performance data
supplier_data = {
    'Supplier': ['Supplier_X', 'Supplier_Y'],
    'On_Time_Delivery_Rate': [0.95, 0.85],
    'Quality_Rating': [4.5, 3.8],
    'Cost_Per_Unit': [10, 12]
}
supplier_df = pd.DataFrame(supplier_data)

# Enhance logistics data with more details
logistics_data = {
    'Route': ['Route_A', 'Route_B', 'Route_C'],
    'Distance_km': [100, 150, 200],
    'Traffic_Level': ['Low', 'Medium', 'High'],
    'Fuel_Cost_Per_km': [1.2, 1.5, 1.8],
    'Delivery_Time_hours': [2, 3, 4],
    'Carrier': ['Carrier_1', 'Carrier_2', 'Carrier_3']
}
logistics_df = pd.DataFrame(logistics_data)




In [ ]:
sales_df.head()


,Date,Product,Sales
0,2023-01-01,Product_A,23
1,2023-01-02,Product_A,15
2,2023-01-03,Product_A,21
3,2023-01-04,Product_A,25
4,2023-01-05,Product_A,15


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Prepare data for demand forecasting
sales_df['DayOfYear'] = sales_df['Date'].dt.dayofyear

# Encode products as dummy variables
sales_df = pd.get_dummies(sales_df, columns=['Product'], drop_first=True)

# Split data into features and target
X = sales_df[['DayOfYear', 'Product_Product_B', 'Product_Product_C']]
y = sales_df['Sales']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')


Mean Absolute Error: 3.64


In [ ]:
def optimize_route(logistics_df):
    # Simple heuristic: choose the route with the lowest traffic level and shortest distance
    traffic_levels = {'Low': 1, 'Medium': 2, 'High': 3}
    logistics_df['Traffic_Score'] = logistics_df['Traffic_Level'].map(traffic_levels)
    optimized_route = logistics_df.sort_values(by=['Traffic_Score', 'Distance_km']).iloc[0]
    return optimized_route

optimized_route = optimize_route(logistics_df)
print(f'Optimized Route: {optimized_route["Route"]}')


Optimized Route: Route_A


In [ ]:
def optimal_inventory_level(product, inventory_df):
    row = inventory_df[inventory_df['Product'] == product].iloc[0]
    optimal_level = row['Current_Stock'] - row['Reorder_Level']
    return optimal_level

# Example
optimal_level_A = optimal_inventory_level('Product_A', inventory_df)
print(f'Optimal inventory level for Product_A: {optimal_level_A}')


Optimal inventory level for Product_A: 300


In [ ]:
def supplier_performance(supplier, supplier_df):
    row = supplier_df[supplier_df['Supplier'] == supplier].iloc[0]
    performance_score = (row['On_Time_Delivery_Rate'] * 0.5 + row['Quality_Rating'] * 0.3 + (10 / row['Cost_Per_Unit']) * 0.2)
    return performance_score

# Example
performance_X = supplier_performance('Supplier_X', supplier_df)
print(f'Supplier_X performance score: {performance_X:.2f}')


Supplier_X performance score: 2.02


In [20]:
import json



qa_pairs = [
    {"question": "What is the forecast demand for Product_A?", "answer": "The forecasted demand for Product_A is 300 units."},
    {"question": "How can we optimize our delivery route?", "answer": "The optimized route is Route_A, with a distance of 100 km and traffic level Low."},
    {"question": "What is the optimal inventory level for Product_B?", "answer": "The optimal inventory level for Product_B is 200 units."},
    {"question": "How is Supplier_X performing?", "answer": "Supplier_X has an on-time delivery rate of 95% and a quality rating of 4.5."},
    {"question": "When should we reorder Product_C?", "answer": "Reorder Product_C when stock reaches 300 units, considering a lead time of 3 days."}
]

# Expand the dataset to include more detailed questions and answers
for product in ["Product_B", "Product_C"]:
    qa_pairs.append({
        "question": f"What is the forecast demand for {product}?",
        "answer": f"The forecasted demand for {product} is {sales_df[product].mean()} units."
    })
    qa_pairs.append({
        "question": f"What is the current stock level of {product}?",
        "answer": f"The current stock level of {product} is {inventory_df[inventory_df['Product'] == product]['Current_Stock'].values[0]} units."
    })
    qa_pairs.append({
        "question": f"What is the reorder level for {product}?",
        "answer": f"The reorder level for {product} is {inventory_df[inventory_df['Product'] == product]['Reorder_Level'].values[0]} units."
    })
    qa_pairs.append({
        "question": f"What is the lead time for {product}?",
        "answer": f"The lead time for {product} is {inventory_df[inventory_df['Product'] == product]['Lead_Time_Days'].values[0]} days."
    })


KeyError: 'Product_B'

In [ ]:
def create_prompt(question, qa_pairs):
    prompt = "Answer the following supply chain optimization questions:\n\n"
    for pair in qa_pairs:
        prompt += f"Q: {pair['question']}\nA: {pair['answer']}\n\n"
    prompt += f"Q: {question}\nA:"
    return prompt


In [18]:
import os
import mistralai
from mistralai.client import MistralClient
from google.colab import userdata
# Set up Mistral.ai API key
os.environ["MISTRAL_API_KEY"]=userdata.get("MISTRAL")  # Replace with your actual API key
mistral_api_key = os.getenv("MISTRAL_API_KEY")

client = MistralClient(api_key=mistral_api_key)

# Function to generate embeddings using Mistral.ai
def generate_embeddings(text):
    response = client.embeddings(
        input=text,
        model="mistral-embed"  # Choose an appropriate model for your use case
    )
    return response.data[0].embedding

def find_most_relevant_answer(question_embedding, qa_pairs):
    best_match = None
    best_similarity = -1
    for pair in qa_pairs:
        answer_embedding = generate_embeddings(pair['answer'])
        similarity = np.dot(question_embedding, answer_embedding) / (np.linalg.norm(question_embedding) * np.linalg.norm(answer_embedding))
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = pair['answer']
    return best_match

def answer_question(question, qa_pairs):
    question_embedding = generate_embeddings(question)
    answer = find_most_relevant_answer(question_embedding, qa_pairs)
    return answer

# # Example questions
# questions = [
#     "What is the forecast demand for Product_A?",
#     "How can we optimize our delivery route?",
#     "What is the optimal inventory level for Product_B?",
#     "How is Supplier_X performing?",
#     "When should we reorder Product_C?",
#     "What is the lead time for Product_C?",
#     "Which supplier has the best performance?",
#     "How can we reduce our logistics costs?",
#     "What is the reorder point for Product_A?"
# ]

# for question in questions:
#     answer = answer_question_llm(question, qa_pairs)
#     print(f'Q: {question}\nA: {answer}\n')

# User interaction loop
while True:
    user_question = input("Ask a supply chain optimization question (or type 'exit' to quit): ")
    if user_question.lower() == 'exit':
        break
    answer = answer_question(user_question, qa_pairs)
    print(f'Answer: {answer}\n')



Ask a supply chain optimization question (or type 'exit' to quit): What is the forecast demand for Product_b
Answer: The forecasted demand for Product_A is 23.4 units.

Ask a supply chain optimization question (or type 'exit' to quit): What is the forecast demand for Product_B
Answer: The forecasted demand for Product_A is 23.4 units.

Ask a supply chain optimization question (or type 'exit' to quit): EXIT


In [15]:
pip install mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
